<h1>  <center> SCRAPING DE GROUPE PUBLIC FACEBOOK</center> </h1>


Cette partie sera consacrée à l'étude de la **E-REPUTATION** d'une marque/entreprise/personnalité (selon le besoin) sur les réseaux sociaux ; en particulier sur un **groupe FACEBOOK**.


La e-réputation, c'est tout simplement l'idée ou l'opinion qu'a le public envers une personne, une entreprise, un groupe, ou une organisation sur le net. Il est donc important de connaître les plateformes externes vers lesquelles vos clients ou prospects pourraient potentiellement se tourner pour y exprimer leurs sentiments vis-à-vis de vos différents services.

Un lieu sûr, les réseaux sociaux :  Facebook, WhatsApp, Twitter, Instagram, ... 


. | .
- | - 
![alt](images/ereputation_1.png) | ![alt](images/ereputation_2.png)


Là, Tout est relax !!! les internautes peuvent se plaindre comme ils le souhaitent;    
Attention !!! Il vous faudra porter une attention particulière aux influenceurs. Ces derniers peuvent influencer la perception de ses abonnés ou fans sur votre marque/organisation/marque.



L'idéal serait donc de parvenir à surveiller de prêt, sur toutes les plateformes en lignes, l'opinion du public sur votre entreprise.  
  
  
## Mais comment y parvenir ?


Des entités optent pour le recrutement de ressources humaines dédiées spécialement à la réalisation de ces tâches.  

Imaginez-vous à la place de ces personnes😲😲😲.   
Rester scotché devant son écran attendant qu'un internaute porte un jugement pour/contre votre entité sur une plateforme en ligne !!!    
Et là, je vous parle d'une plateforme; comment feriez-vous pour les autres plateformes 🤔🤔🤔?    
 Fastidieux non 🥱🥱?    



Mais RELAX😉😉😉, fort heureusement, des outils de scraping existent pour collecter automatiquement les traces laissées par les internautes sur le net.

Vous pourrez, grâce à ces outils, parcourir plusieurs sites et récupérer les informations souhaitées dans un laps de temps.


#### Bref!!! Passons aux choses sérieuses à présent!

# Collecte du groupe UCAEVD

Dans la suite de ce notebook, nous allons :

- utiliser un driver Selenium le scraping,

- nous connecter au groupe Facebook public **Un Consommateur Averti En Vaut Deux**  via son URL  

- Renseigner nos informations de connexion au besoin

- Scroler la page ouverte à une profondeur souhaitée pour atteindre plus de publications

- Collecter toutes les publications disponibles sur la page chargée (Nous pouvons éventuellement collecter les commentaires)   

- Traiter les données collectées puis les structurer et les insérer dans un DataFrame ou un fichier .



Ensuite, nous pourrons :

- Effectuer des filtre sur les données selon des mots clés pour ressortir les publications mentionnant ces mots clés

- Notificier en temps réel (ou quasi temps réel) les nouvelles publications détectées par mail, SMS, sur une Appli, ...




# I. PHASE DE COLLECTE

## I.1. Importation des librairies

In [2]:
import numpy as np
import pandas as pd
import time
from datetime import datetime

# 
import sys
import os

# Importation de selenium
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

import dateparser

In [1]:
!pip install dateparser

## I.2. Connexion au driver Selenium et à la page du groupe UCAEVD

In [3]:
# service = webdriver.firefox.service.Service(executable_path=gecko_path, port=port)
def connexion_to_group(url, driver_location) :
    
    options = Options()
    # Specifiez le chemin de votre Firefox
    options.binary_location = driver_location
    driver = webdriver.Firefox(options=options)
    time.sleep(2)
    driver.get(url)
    #time.sleep(5)
    
    return driver

In [4]:
driver_location = "C:/Program Files/Mozilla Firefox/firefox.exe"
url = 'https://m.facebook.com/groups/consommateurs.abidjan/'

In [7]:
driver = connexion_to_group(url, driver_location)

## 3. Scroller une page

### 3.1. Cas général

In [8]:
## Fonction de SCROLLING DE LA PAGE 
def scroll_down_page(nb_iter = 3, wait_time = 5):
#     driver.refresh()
    i = 0
    while i <nb_iter:
        i+=1
        print(i)
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(wait_time)
        except:
            break

In [9]:
driver.refresh()
time.sleep(3)
scroll_down_page(nb_iter = 3, wait_time = 5)

1
2
3


Oups !!!

La méthode de scrolling d'une page de façon générale fonctionne, mais pas comme on le souhaite ici.

Facebook a renvoyé un popup pour nous demander de nous authentifier. Ce qui a empêché le scroll de continuer son travail.

![alt](images/scroll_page.png) 

Mais RELAX, on peut contourner ce imprévu dans le cas particulier suivant.

Dans la section suivante, nous allons cliquer sur la croix entourée en rouge sur l'image ci-dessous et poursuivre le scrolling.

Pour mettre cela en visibilité, nous allons rouvrir un autre navigateur et nous connecter ensuite au groupe puis scroller la page du groupe.

In [10]:
driver.quit()

### I.3.2. Cas Particulier

In [20]:
def afficher_plus_de_pubs(nb_iter = 3, wait_time = 5):
#     driver.refresh()
    nb = nb_iter - 1
    i = 0
    while i <nb_iter:
        i+=1
        
        try:
            ## Afficher plus de publications
            # driver.find_element(By.CSS_SELECTOR, ".touchable.primary").click()
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(wait_time)
            print(i)
            try :
                driver.find_element(By.ID, "popup_xout").click()
            except :
                pass
        except:
            break
    if i == 1 :
        afficher_plus_de_pubs(nb_iter = nb, wait_time = 5)

In [21]:
#driver_location = "put_path_to_your_driver_location"
driver_location = "C:/Program Files/Mozilla Firefox/firefox.exe"
url = 'https://m.facebook.com/groups/consommateurs.abidjan'

driver = connexion_to_group(url, driver_location)


In [22]:
driver.refresh()
afficher_plus_de_pubs(nb_iter = 5, wait_time = 5)

## I.4. Obtenir toutes les publications/posts

In [18]:
## Get all publications available on the page

def get_all_publications() : 
    # Access to all posts stories container
    publications = driver.find_element(By.ID, "m_group_stories_container")

    # Get all posts relative to articles in stories container
    publications = publications.find_elements(By.TAG_NAME, "article")
    
    return publications

In [19]:
# Collect publications
publications = get_all_publications()


NoSuchElementException: Message: Unable to locate element: [id="m_group_stories_container"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:133:16


In [31]:
len(publications)

120

In [24]:
publications[:5]

[<selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="d23ffdc5-75d2-4621-94e2-de96b133a816")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="a1ab3b7a-2767-4a8b-aaad-11b91adf9c33")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="b43f343e-99a3-4e02-ab14-79a980af5383")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="f93b6a00-3620-4e20-88e0-7e839492c90e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="e8b3d93a-7ee5-4642-81b5-26edae3bd94a")>]

### I.4.1. Obtenir les ID des posts

In [32]:
pub0 = publications[0]
pub0.get_attribute("data-store")

'{"linkdata":"encrypted_tracking_data.0AY_G5Ko6L4yfJOrbXdyfL289fBzzqfhkUJFfse4OVcVS6XxVzwMHCXR0nx7WQWcoD1TJ-wv-ZrAcmgLe-IMj80QcUYX2tsq0lHpbRiDFebWbspv1V-mFyY2C9kT5AdujIso_IwzxtTXSfvs_NoCflqAY0vpFKpjskOdtxtZYuyR6TWB-dXCY1M_EydZiDQearjWTVzA7gJcM8VddjcFDtRWVvHEG_eVsCj9d8bBPBs1-m3kCRXcBD8JogPJNbaZzD82Ji91LHB3iwDCyANYIbVPPtubiR1pmDomOL3aGzdmNoQeWUIF7lVGgz0-il0XvHn6mY7NqYN3MAPTurF93Sxqohe_nZkvJTlWZthbBvOZ0H7z_3rxYrEE2H2GeAasiK84yJBwGpy-bcTlcKcHHsn4H0n6nCKZVF7EiTyqk9BR33iY2Ppyz-m9pdm8PjyxSIz3JMxlh0L_3wrgrFgELPdJI-Ofvq39y0jLw8lze0ZnrUA7P9_4GlQRTKmwR4Tuypy_Hn-jfrz7JoysLZK-RS_q3lxKhMJDeEvkQvCiK-NaVTD1TpVgi2Pb_nqDfC5P9dQH_xJxNdEG9RGDPaKCBcB8VyxDF7RXVZTlNlXmbuoxmpMA_MCG1NvxzpMXp_7Z_1kQTJGg4LJ-5lQQkxmQ951EGO6MvlwXGGuTB47-oiaLufu0Xc7qoFY-14VIQyfC3LxwmOUc1GDMmaEmpwaUAf3aUiH_JoUTJmdCfIljTx-DYMYASgHlg5E7xS92pL1ZzOBUOWS9bmqg91wfeMvRG7UTHgcSGLfcVoYeV4lOhkusVLDNtXNNc3YQ4iGdLFRa0VJS86DGZf9lwn2rM55ai6naKP2_VpEr0FUDI8ravvk8Kkt7KdbP6JgKCA1gwB--cZ8P8zTJvGmOcAGV3Qv2-2YVA6_TvMNOZNO-jE4O4Vu3GNUR42xrxeKidUMHQK4UpB

In [23]:
## Get id_post
def get_id_post(post) :
    
    id_post = post.get_attribute("data-store")
    id_post = id_post.replace("null", "None")
    # Conversion d'un STRING en DICTIONNAIRE
    id_post_dict = eval(id_post)
    id_post = id_post_dict['share_id'] 
   
    return id_post

In [33]:
[get_id_post(post) for post in publications[:10]]

[5952737848095254,
 5952739368095102,
 5948593565176349,
 5948188328550206,
 5950219581680414,
 5951945208174518,
 5949569345078771,
 5949986491703723,
 5951429174892788,
 5952275541474818]

### I.4.2. Obtenir les liens des posts

In [34]:
## Get the post link : post_link
def get_post_link(url, id_post) :
    
    permalink = 'permalink/'
    post_link = url + permalink + str(id_post)
    
    if 'https://m' in url :
        url_post = url.replace("https://m", "https://www")
        post_link = url_post + permalink + str(id_post)
    
    return post_link

In [35]:
url = 'https://m.facebook.com/groups/consommateurs.abidjan/'
id_posts = [get_id_post(post) for post in publications[:10]]

posts_links = [get_post_link(url, id_post) for  id_post in id_posts]
posts_links

['https://www.facebook.com/groups/consommateurs.abidjan/permalink/5952737848095254',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5952739368095102',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5948593565176349',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5948188328550206',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5950219581680414',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5951945208174518',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5949569345078771',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5949986491703723',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5951429174892788',
 'https://www.facebook.com/groups/consommateurs.abidjan/permalink/5952275541474818']

### I.4.3. Obtenir les dates des posts

In [36]:
### Get date post infos
def info_date_post(post) :
    
    info_date_post = post.find_element(By.TAG_NAME, "abbr")
    info_date_post = info_date_post.text
    return info_date_post

In [37]:
[info_date_post(post) for post in publications[:10]]

['52 min',
 '53 min',
 '20 h',
 'Hier, à 08:14',
 '6 h',
 '1 h',
 '6 h',
 '5 h',
 '7 h',
 '51 min']

In [38]:
liste_dates_test =  [info_date_post(post) for post in publications[:10]]
[dateparser.parse(str_val) for str_val in liste_dates_test]

[datetime.datetime(2023, 1, 28, 15, 45, 39, 201686),
 datetime.datetime(2023, 1, 28, 15, 44, 39, 213653),
 datetime.datetime(2023, 1, 27, 20, 37, 39, 216651),
 datetime.datetime(2023, 1, 27, 8, 14),
 datetime.datetime(2023, 1, 28, 10, 37, 39, 362449),
 datetime.datetime(2023, 1, 28, 15, 37, 39, 363442),
 datetime.datetime(2023, 1, 28, 10, 37, 39, 364447),
 datetime.datetime(2023, 1, 28, 11, 37, 39, 366434),
 datetime.datetime(2023, 1, 28, 9, 37, 39, 367431),
 datetime.datetime(2023, 1, 28, 15, 46, 39, 371911)]

#### Cleaning des dates

In [40]:
##   Convert the string date format to datetime 
import dateparser
def transform_string_date(str_val):
    str_val = str_val.lower()
    date_ = dateparser.parse(str_val)
    
    if 'instant' in str_val:
            return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    return date_.strftime("%Y-%m-%d %H:%M:%S")

## Clean the date and convert it in "%Y-%m-%d %H:%M:%S", "%Y-%m-%d" and "%H:%M:%S" format
def clean_date(thedate):
    thedate = datetime.strptime(thedate, "%Y-%m-%d %H:%M:%S")
    return thedate, thedate.date(), thedate.time()


In [47]:
transform_string_date('27/mai/2022    , 17:22')

'2022-05-27 17:22:00'

In [48]:
print(clean_date(transform_string_date('27 mai, 17:22')))

(datetime.datetime(2023, 5, 27, 17, 22), datetime.date(2023, 5, 27), datetime.time(17, 22))


### I.4.4. Obtenir les contenus textuels des posts

In [53]:
## Get post textual content
def get_pub_text(post):
    
    text_pub = ''
    
    try :
        text_pub = post.find_element(By.CSS_SELECTOR, "._5rgt._5nk5")
        
#         text_pub.find_element_by_link_text("Plus").click()
        
    except :
        pass

    text_pub = post.find_element(By.CSS_SELECTOR, "._5rgt._5nk5")
    text = text_pub.text
#     _5rgt _5nk5
                                                 
    return text

In [54]:
[get_pub_text(post) for post in publications[:10]]

["#transpers #csoa\nAh la Sotra!jusqu'à quand vous allez nous maltraiter dans notre argent ?\nJ'aimerais comprendre la ligne express 205 Angré-Plateau serait-elle devenue semblable au 82?\nNormalement pour un tarif de 500f la prestation est de veiller au fait qu'une fois toutes les places assises sont occupées,le machiniste ne devrait plus vendre… Plus",
 '#textile\nBonjour à tous merci de me recommander un bon couturier dans la zone de Yopougon (toit rouge si possible) très fort en modèle pagne pour femmes.\nMerci à tous !',
 "#textile\nBonjour à tous, je recherche une tenue de la gendarmerie nationale pour mardi gras ou quelqu'un susceptible de le faire.\nMerci à vous",
 '#transpers #csoa\nEn espérant que les dirigeants de la SOTRA et ceux qui gèrent la ligne 775(château-chu de treichville) puissent voir ce poste.\nFranchement on en a vraiment marre.\nChaque jour on se réveille à… Plus',
 '#textile\nBsr à tous,\nSvp besoin de votre aide, je cherche ce modèle pareil sur la photo.\nMer

### I.4.5. Obtenir les infos sur les Htags


In [55]:
## Get htags infos
def get_htags_info(post) :
    
    htag_pub_desc = None
    htag_link = None
    
    ### 2.2. Get htags in publication content : htag_pub_desc
    try :
        htag_pub_desc = post.find_elements(By.CLASS_NAME, "_5ayu")
        htag_pub_desc = "|".join([htag.text for htag in htag_pub_desc])
        htag_pub_desc
    except :
        pass
    
    ### 2.3. Get htags link : htag_link
    try :
        htag_link = post.find_elements(By.CSS_SELECTOR, "._5ayv._qdx")
        htag_link = "|".join([htag.get_attribute("href") for htag in htag_link])
        htag_link
    except :
        pass

    return htag_pub_desc, htag_link

In [56]:
[get_htags_info(post) for post in publications[:5] ]

[('transpers|csoa',
  'https://m.facebook.com/hashtag/transpers?refid=18&_ft_=encrypted_tracking_data.0AY_G5Ko6L4yfJOrbXdyfL289fBzzqfhkUJFfse4OVcVS6XxVzwMHCXR0nx7WQWcoD1TJ-wv-ZrAcmgLe-IMj80QcUYX2tsq0lHpbRiDFebWbspv1V-mFyY2C9kT5AdujIso_IwzxtTXSfvs_NoCflqAY0vpFKpjskOdtxtZYuyR6TWB-dXCY1M_EydZiDQearjWTVzA7gJcM8VddjcFDtRWVvHEG_eVsCj9d8bBPBs1-m3kCRXcBD8JogPJNbaZzD82Ji91LHB3iwDCyANYIbVPPtubiR1pmDomOL3aGzdmNoQeWUIF7lVGgz0-il0XvHn6mY7NqYN3MAPTurF93Sxqohe_nZkvJTlWZthbBvOZ0H7z_3rxYrEE2H2GeAasiK84yJBwGpy-bcTlcKcHHsn4H0n6nCKZVF7EiTyqk9BR33iY2Ppyz-m9pdm8PjyxSIz3JMxlh0L_3wrgrFgELPdJI-Ofvq39y0jLw8lze0ZnrUA7P9_4GlQRTKmwR4Tuypy_Hn-jfrz7JoysLZK-RS_q3lxKhMJDeEvkQvCiK-NaVTD1TpVgi2Pb_nqDfC5P9dQH_xJxNdEG9RGDPaKCBcB8VyxDF7RXVZTlNlXmbuoxmpMA_MCG1NvxzpMXp_7Z_1kQTJGg4LJ-5lQQkxmQ951EGO6MvlwXGGuTB47-oiaLufu0Xc7qoFY-14VIQyfC3LxwmOUc1GDMmaEmpwaUAf3aUiH_JoUTJmdCfIljTx-DYMYASgHlg5E7xS92pL1ZzOBUOWS9bmqg91wfeMvRG7UTHgcSGLfcVoYeV4lOhkusVLDNtXNNc3YQ4iGdLFRa0VJS86DGZf9lwn2rM55ai6naKP2_VpEr0FUDI8ravvk8Kkt7KdbP6JgKCA1gwB--cZ

### I.4.6. Obtenir les liens des images et leur description

In [65]:
## Get image link and its description
def get_image_infos(post, sep = '|') :
    
    image_link = ''
    image_desc = ''
    
    ## Get image link and its description
    try :
    ## Get image link and its description
        image_link_desc = post.find_element(By.CLASS_NAME, "story_body_container")
        image_link_desc = image_link_desc.find_element(By.CLASS_NAME, "_5rgu._7dc9")
        image_link_desc = image_link_desc.find_elements(By.TAG_NAME, "img")

        ## Image link : image_link
        image_link = sep.join([img_link.get_attribute("src") for img_link in image_link_desc])
        # image_link = " | ".join([img_link.get_attribute("src") for img_link in image_link_desc])

        ## Image description : image_desc
        image_desc = sep.join([img_desc.get_attribute("alt") for img_desc in image_link_desc])
        # image_desc = " | ".join([img_desc.get_attribute("alt") for img_desc in image_link_desc])

    except :
        pass
    
    return image_link, image_desc

In [66]:
[get_image_infos(post, sep = '|') for post in publications[:5] ]

[('', ''),
 ('', ''),
 ('https://scontent.fabj8-1.fna.fbcdn.net/v/t39.30808-6/327422460_1210759189533307_1031513303378751292_n.jpg?stp=cp0_dst-jpg_e15_p240x240_q65&_nc_cat=106&ccb=1-7&_nc_sid=5bac3a&_nc_ohc=8rDfTHoBozcAX9WCbPQ&_nc_ht=scontent.fabj8-1.fna&oh=00_AfBvgybS7I7ASc6kSTib2CUbJI-Gi99hpIWj_329NUzAxA&oe=63D9D199|https://scontent.fabj8-1.fna.fbcdn.net/v/t39.30808-6/327301337_902286900810206_6852749580541192446_n.jpg?stp=cp0_dst-jpg_e15_p240x240_q65&_nc_cat=103&ccb=1-7&_nc_sid=5bac3a&_nc_ohc=OHAK7J5mikQAX-rtH3V&_nc_ht=scontent.fabj8-1.fna&oh=00_AfCfklIgFKuDeaQw46iVCpVykA8qguQiYr_H9USgtZvumw&oe=63DA624C',
  'Peut être une image de 1 personne, position debout et intérieur|Peut être une image de 9 personnes, personnes debout et plein air'),
 ('https://scontent.fabj8-1.fna.fbcdn.net/v/t39.30808-6/326976910_536024958498212_366665613941034206_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=109&ccb=1-7&_nc_sid=5bac3a&_nc_ohc=oFstd4-x8LAAX9yeAXb&_nc_ht=scontent.fabj8-1.fna&oh=00_AfDK70yozwJMmzdfW

## I.5. Récupérer les réactions

Cette section de code permet d'accéder à la zone des réactions effectuées sur les publications postées sur le groupe.

In [73]:
## Get  posts reactions driver element
def reactions_elements(post) :
    
    reactions = []
    try :
        reaction = post.find_element(By.TAG_NAME, "footer").\
        find_element(By.CLASS_NAME, "_2og4.slimLike").\
        find_element(By.TAG_NAME, "a")
        reactions = reaction.find_elements(By.CLASS_NAME, "_28wy")
    except :
        pass
    
    return reactions

In [74]:
[reactions_elements(post) for post in publications[:5] ]

[[<selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="9c8b729c-1389-4948-a5ea-70952fae76f7")>,
  <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="ca087940-2e9a-4af8-93b7-9f93032acba4")>],
 [<selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="e1a5b314-3315-4ad0-9ade-283d0b02a7a6")>,
  <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="f2a8df45-d44f-4a84-a3bc-b554221fa0c5")>],
 [<selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="ab541d43-62c6-49c4-93f0-49f51fc01364")>,
  <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6", element="3aef1255-aaaa-4f0d-8e82-a3eef32655e1")>,
  <selenium.webdriver.remote.webelement.WebElement (session="41d47abc-affd-40b5-9c44-ca0fc89e3cb6"

In [75]:
react0 = [reactions_elements(post) for post in publications[:5] ][0]

In [76]:
print(react0[0].text)
print(react0[1].text)

21 J’aime
10 commentaires


### I.5.1. résumé des réactions

Ce code permet de compter les réactions selon le type de réaction :   
Par exemple : Le nombre de **J'aime**, de **commentaires**, de **partages**, de **Haha**, ...

In [77]:
## Count reaction using type reaction
def keep_number(text) :
    
    return ''.join([nber for nber in text if nber.isdigit()]) 

def count_reactions(liste_text, type_text):
    """
    Reactions type can be for :
        - 'aime' type : J'aime
        - 'commentaire' : commentaire , commentaires
       """   
    if len(liste_text) == 0:
        res = '0'

    for react in liste_text :
        
        if type_text in react :
#             print(react)
            res = keep_number(react)
            break
        else :
            res = '0'
    return res

In [78]:
def keep_number(text) :
    
    return ''.join([nber for nber in text if nber.isdigit()])

In [79]:
text = "J'ai réalisé 1 commentaire sur Facebook et  15 partages hier"
keep_number(text)

'115'

In [94]:
liste_text = ["18 commentaire", "19 partage", "30 j'aime"]

In [95]:
count_reactions(liste_text, type_text = 'commentaire')

'18'

In [96]:
count_reactions(liste_text, type_text = 'partage')

'19'

In [89]:
count_reactions(liste_text, type_text = 'aime')

'30'

### I.5.2. Collecte de toutes les réactions

In [97]:
## Count all reactions
def get_posts_reactions(post) :
    
    nbre_jaime = '0'
    nbre_partage = '0'
    nbre_commentaire = '0'
    
    ## Other indicators 
    nbre_jadore = '0'
    nbre_wouah = '0'
    nbre_haha = '0'
    nbre_grrr = '0'
    nbre_triste = '0'
    nbre_autres_reactions = '0'
    
    try :
        ## Get  posts reactions driver element
        reactions = reactions_elements(post)
        reactions = [react.text for react in reactions]

        ### Count Like : nbre_jaime
        nbre_jaime = count_reactions(liste_text = reactions, type_text = 'aime')

        ### Count comments : nbre_commentaire
        nbre_commentaire = count_reactions(liste_text = reactions, type_text = 'commentaire')

        ### Count Sharing : nbre_partage
        nbre_partage = count_reactions(liste_text = reactions, type_text = 'partage')
    except :
        pass
    result = [nbre_jaime, nbre_commentaire, nbre_partage, nbre_jadore,nbre_wouah,nbre_haha,
              nbre_grrr,nbre_triste,nbre_autres_reactions]
    return result

In [98]:
[get_posts_reactions(post) for post in publications[:5] ]

[['21', '10', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '3', '0', '0', '0', '0', '0', '0', '0'],
 ['285', '175', '14', '0', '0', '0', '0', '0', '0'],
 ['373', '154', '47', '0', '0', '0', '0', '0', '0'],
 ['86', '86', '5', '0', '0', '0', '0', '0', '0']]

# II. PHASE DE STOCKAGE

## II.1. Stockage des données sur les infos générales des posts

In [99]:
## 1. Data author posts Infos
def data_author_posts(posts) :
        
    id_posts = []
    post_links = []
    date_time_posts = []
    date_posts = []
    hours_posts = []
    
    for i, post in enumerate(posts):
        # Vous pouvez activer la ligne ci-dessous pour controler afficher les itérations
        # print("Renseignement des Infos/Auteur post \t{}".format(i))
        
        # Get post ID
        id_post = get_id_post(post)
        id_posts.append(id_post)
        
        # Get Post URL
        post_link = get_post_link(url, id_post)
        post_links.append(post_link)
        
        ## Get dates post
        date_infos = transform_string_date(info_date_post(post))
        date_infos = clean_date(date_infos)
        # Code to develop here
        date_time_post = date_infos[0]
        date_time_posts.append(date_time_post)
        
        # Code to develop here
        date_post = date_infos[1]
        date_posts.append(date_post)
        
        # Code to develop here
        hours_post = date_infos[2]
        hours_posts.append(hours_post)        

    authors_posts_dates = pd.DataFrame(
        data= {'id_post':id_posts, 'post_link':post_links,'date_time_post':date_time_posts,
                      'date_post':date_posts, 'hours_post':hours_posts 
                      }, 
        columns = ['id_post', 'post_link','date_time_post','date_post', 'hours_post']
         )
    
    return authors_posts_dates

In [100]:
author_post_data = data_author_posts(publications)
author_post_data.head()


,id_post,post_link,date_time_post,date_post,hours_post
0,5952737848095254,https://www.facebook.com/groups/consommateurs....,2023-01-28 16:58:46,2023-01-28,16:58:46
1,5952739368095102,https://www.facebook.com/groups/consommateurs....,2023-01-28 16:57:46,2023-01-28,16:57:46
2,5948593565176349,https://www.facebook.com/groups/consommateurs....,2023-01-27 21:50:47,2023-01-27,21:50:47
3,5948188328550206,https://www.facebook.com/groups/consommateurs....,2023-01-27 08:14:00,2023-01-27,08:14:00
4,5950219581680414,https://www.facebook.com/groups/consommateurs....,2023-01-28 11:50:47,2023-01-28,11:50:47


## II.2. Stockage des données des contenus des posts

In [101]:
## 2. data_content_post
def data_contents_posts(posts):
    id_posts = []
    text_pub = []
    htag_pub_desc = []
    htag_links = []
    
    for i, post in enumerate(posts):
        # Vous pouvez activer la ligne ci-dessous pour controler afficher les itérations        #print("Contenu du post \t{}".format(i))
        
        # Get post ID
        id_post = get_id_post(post)
        id_posts.append(id_post)
        
        ## Textual content of the publication
        text_post = get_pub_text(post)
        text_pub.append(text_post)
        
        ## Hatgs in the post 
        htag_desc = get_htags_info(post)[0]
        htag_pub_desc.append(htag_desc)
        
        ## Hatg link in the post
        htag_link = get_htags_info(post)[1]
        htag_links.append(htag_link)

    # Donnees des Auteurs et dates de posts
    contents_posts = pd.DataFrame(data= { 'id_post' : id_posts,
        'text_pub' : text_pub, 'htag_pub_desc' : htag_pub_desc,
      'htag_link' : htag_links  
    }, columns = ['id_post','text_pub', 'htag_pub_desc',  'htag_link'])
    
    return contents_posts

In [103]:
content_post_data = data_contents_posts(publications)
content_post_data.head(10)

,id_post,text_pub,htag_pub_desc,htag_link
0,5952737848095254,#transpers #csoa\nAh la Sotra!jusqu'à quand vo...,transpers|csoa,https://m.facebook.com/hashtag/transpers?refid...
1,5952739368095102,#textile\nBonjour à tous merci de me recommand...,textile,https://m.facebook.com/hashtag/textile?refid=1...
2,5948593565176349,"#textile\nBonjour à tous, je recherche une ten...",textile,https://m.facebook.com/hashtag/textile?refid=1...
3,5948188328550206,#transpers #csoa\nEn espérant que les dirigean...,transpers|csoa,https://m.facebook.com/hashtag/transpers?refid...
4,5950219581680414,"#textile\nBsr à tous,\nSvp besoin de votre aid...",textile,https://m.facebook.com/hashtag/textile?refid=1...
5,5951945208174518,QG Abidjan - Riviera Ciad\nSi vous cherchez un...,,
6,5949569345078771,Bonsoir !!!\n\nJe recherche ce genre de mèche....,,
7,5949986491703723,#agroalim\nBonsoir chers tous..\nAdmis svp .. ...,agroalim,https://m.facebook.com/hashtag/agroalim?refid=...
8,5951429174892788,#restau Bonjour a tous. Je recherche ces petit...,restau,https://m.facebook.com/hashtag/restau?refid=18...
9,5952275541474818,#Maroq\nBonjour chers tous.\nPouvez-vous m'aid...,Maroq,https://m.facebook.com/hashtag/maroq?refid=18&...


## II.3. Stockage données des images dans les posts

In [104]:
## 3. data image in post
def images_posts_info(posts, sep = '|'):
    id_posts = []
    images_desc = []
    images_link = []
    
    for i, post in enumerate(posts):
        # Vous pouvez activer la ligne ci-dessous pour controler afficher les itérations
        #print("Collecte des images du post \t{}".format(i))
        
        # Get post ID
        id_post = get_id_post(post)
        id_posts.append(id_post)
        
        image_desc = get_image_infos(post, sep)[1]
        images_desc.append(image_desc)
        
        image_link = get_image_infos(post, sep)[0]
        images_link.append(image_link)
#         print("Image link \t{}".format(image_link))
    # Infos sur les images
    images_desc_links = pd.DataFrame(data= {'id_post' : id_posts,
        'image_desc' : images_desc, 'image_link' : images_link
    }, columns = ['id_post','image_desc', 'image_link'])
    return images_desc_links

In [105]:
sep = "|"
images_post_data = images_posts_info(publications, sep)
images_post_data.head()

,id_post,image_desc,image_link
0,5952737848095254,,
1,5952739368095102,,
2,5948593565176349,"Peut être une image de 1 personne, position de...",https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...
3,5948188328550206,Peut être une image de texte qui dit ’sotro So...,https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...
4,5950219581680414,"Peut être une image de 1 personne, position de...",https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...


## II.4. Stockage données des réactions

In [106]:
## 4. Reactions data
def data_reactions(posts) :
    liste = ['id_post','nbre_jaime','nbre_commentaire', 'nbre_partage','nbre_jadore', 'nbre_wouah', 
             'nbre_haha', 'nbre_grrr', 'nbre_triste','nbre_autres_reactions' ]
    
    nb_reactions = []
    
    for i, post in enumerate(posts):
        # Vous pouvez activer la ligne ci-dessous pour controler afficher les itérations
        #print("Collecte des reactions \t{}".format(i))
        
        # Get post ID
        id_post = get_id_post(post)
        id_posts = list([id_post]) 
        
        reacts_elts = get_posts_reactions(post)
        reacts_elts = id_posts + reacts_elts
        nb_reactions.append(reacts_elts)
        
    data = pd.DataFrame(nb_reactions, columns=liste)
    
    return data

In [107]:
reactions_post_data = data_reactions(publications)
reactions_post_data.head()

,id_post,nbre_jaime,nbre_commentaire,nbre_partage,nbre_jadore,nbre_wouah,nbre_haha,nbre_grrr,nbre_triste,nbre_autres_reactions
0,5952737848095254,21,10,0,0,0,0,0,0,0
1,5952739368095102,4,3,0,0,0,0,0,0,0
2,5948593565176349,285,175,14,0,0,0,0,0,0
3,5948188328550206,373,154,47,0,0,0,0,0,0
4,5950219581680414,86,86,5,0,0,0,0,0,0


## II.5. Consolidation des données

In [108]:
## 5. Join all informations on collected publications 
def post_data_reacts_content(author_df, content_df, image_df, reacts_df) :
    
    posts_data_reacts = pd.merge(author_df, content_df, how = 'inner', on = 'id_post')
    posts_data_reacts = pd.merge(posts_data_reacts, image_df, how = 'inner', on = 'id_post')
    posts_data_reacts = pd.merge(posts_data_reacts, reacts_df, how = 'inner', on = 'id_post')
    posts_data_reacts = posts_data_reacts[posts_data_reacts['id_post'].notnull()]
    posts_data_reacts = posts_data_reacts.drop_duplicates(['id_post'])
    posts_data_reacts['text_pub'] = posts_data_reacts['text_pub'].astype(str)
    posts_data_reacts = posts_data_reacts[posts_data_reacts['text_pub']!='']
    
    return posts_data_reacts

In [109]:
posts_data_reacts = post_data_reacts_content(author_post_data, 
                                             content_post_data,
                                             images_post_data,
                                             reactions_post_data)
posts_data_reacts = posts_data_reacts[posts_data_reacts['text_pub']!='']

posts_data_reacts.head()

,id_post,post_link,date_time_post,date_post,hours_post,text_pub,htag_pub_desc,htag_link,image_desc,image_link,nbre_jaime,nbre_commentaire,nbre_partage,nbre_jadore,nbre_wouah,nbre_haha,nbre_grrr,nbre_triste,nbre_autres_reactions
0,5952737848095254,https://www.facebook.com/groups/consommateurs....,2023-01-28 16:58:46,2023-01-28,16:58:46,#transpers #csoa\nAh la Sotra!jusqu'à quand vo...,transpers|csoa,https://m.facebook.com/hashtag/transpers?refid...,,,21,10,0,0,0,0,0,0,0
1,5952739368095102,https://www.facebook.com/groups/consommateurs....,2023-01-28 16:57:46,2023-01-28,16:57:46,#textile\nBonjour à tous merci de me recommand...,textile,https://m.facebook.com/hashtag/textile?refid=1...,,,4,3,0,0,0,0,0,0,0
2,5948593565176349,https://www.facebook.com/groups/consommateurs....,2023-01-27 21:50:47,2023-01-27,21:50:47,"#textile\nBonjour à tous, je recherche une ten...",textile,https://m.facebook.com/hashtag/textile?refid=1...,"Peut être une image de 1 personne, position de...",https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...,285,175,14,0,0,0,0,0,0
3,5948188328550206,https://www.facebook.com/groups/consommateurs....,2023-01-27 08:14:00,2023-01-27,08:14:00,#transpers #csoa\nEn espérant que les dirigean...,transpers|csoa,https://m.facebook.com/hashtag/transpers?refid...,Peut être une image de texte qui dit ’sotro So...,https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...,373,154,47,0,0,0,0,0,0
4,5950219581680414,https://www.facebook.com/groups/consommateurs....,2023-01-28 11:50:47,2023-01-28,11:50:47,"#textile\nBsr à tous,\nSvp besoin de votre aid...",textile,https://m.facebook.com/hashtag/textile?refid=1...,"Peut être une image de 1 personne, position de...",https://scontent.fabj8-1.fna.fbcdn.net/v/t39.3...,86,86,5,0,0,0,0,0,0


### II.6. Stockage en format Parquet

Pour stocker vos fichiers textuels (surtout provenant des réseaux sociaux où les internautes qui écrivent un peu de tout à volonté), il est préférable d'utiliser le stockage en format **parquet** , **JSON**, ...

Pour le stockage en format **parquet**, vous aurez besoin d'intaller le package **pyarrow** en faisant ``pip install pyarrow``.

In [131]:
from datetime import datetime
def stored_post_data_to_parquet(data, dossier, file_name = 'posts_data_reacts') :
    """
    dossier : storage directory ;
    type_infos : {'posts', 'commentaires'} ;
    group_name : the name or the abbreviation of the collected group ;
    file_name : the file name (without extension); 

    """
    data = data[data['text_pub']!='']
    now = str(datetime.now())[:19]
    # ile_date : the datetime of collect (the started time of collect) in format : YYYY-mm-dd_HH_MM_SS
    file_date = '_'.join(now.split()).replace(':', '_')
    
    os.makedirs(dossier, exist_ok=True)
    stored_path = f"{dossier}\\{file_name}_{file_date}.parquet"
    data.to_parquet(stored_path, "pyarrow")


In [119]:
!pip install pyarrow

In [138]:
# dossier

'c:\\Users\\Utilisateur\\Documents\\FORMATIONS\\WEB_SCRAPING\\PRATIQUES_WEB_SCRAPING\\SCRAPING_DYNAMIQUE\\DATA'

In [139]:
dossier = str(os.getcwd()) + "\\DATA"
# group_name = "ucaevd"
stored_post_data_to_parquet(data = posts_data_reacts,
                            dossier = dossier,
                            file_name = 'posts_data_reacts')

### II.7. Stockage en CSV

Pour des données textuelles, il est conseillé d'éviter le stockage en CSV pour structurer votre ficher du fait des séparateurs qui peuvent être utilisés sdans les contenus textuels, autant que le souhaite l'auteur.

In [143]:
## 7. Storage collected data
def stored_post_data_to_csv(data, dossier, 
                            file_name = 'posts_data_reacts',
                            sep = "|") :
    """
    dossier : storage directory ;
    type_infos : {'posts', 'commentaires'} ;
    group_name : the name or the abbreviation of the collected group ;
    file_name : the file name (without extension); 

    """
    data = data[data['text_pub']!='']
    from datetime import datetime
    now = str(datetime.now())[:19]
    # ile_date : the datetime of collect (the started time of collect) in format : YYYY-mm-dd_HH_MM_SS
    file_date = '_'.join(now.split()).replace(':', '_')
    
    os.makedirs(dossier, exist_ok=True)
    stored_path = f"{dossier}\\{file_name}_{file_date}.csv"
    data.to_csv(stored_path,  encoding = 'utf-8',
                         index = False, sep = sep)

In [141]:
posts_data_reacts.to_csv()

In [142]:
# dossier

'c:\\Users\\Utilisateur\\Documents\\FORMATIONS\\WEB_SCRAPING\\PRATIQUES_WEB_SCRAPING\\SCRAPING_DYNAMIQUE\\DATA'

In [144]:
dossier = str(os.getcwd()) + "\\DATA"
stored_post_data_to_csv(data = posts_data_reacts,
                            dossier = dossier,
                            file_name = 'posts_data_reacts',
                            sep = "@")

In [57]:
os.getcwd()

'c:\\Users\\Utilisateur\\Documents\\FORMATIONS\\WEB_SCRAPING\\PRATIQUES_WEB_SCRAPING\\SCRAPING_DYNAMIQUE'

In [59]:
os.chdir(os.getcwd())
os.makedirs("./DATA01", exist_ok=True)

Ici, le séparateur utilisé est ``"@"``. Un problème de structuration des données peut suvenir si ``"@"`` est utilisé dans le contenu d'un post.

Par exemple la chaine ``surname.name@inphb.ci`` faussera la structure des champs dans les données stockées.




## III. APPLICATIONS DES FONCTIONS DE COLLECTE

Génial !!!

Vous êtes parvenus à l'étape d'exécution de toutes les fonctions établies pour scraper les publication FACEBOOK d'un groupe public via le lien de la version mobile.

Maintenant, vous pouvez faire l'une des actions suivantes :   

- 1. Action N°1
    
    - Redémarrer tout le notebook  
    - Exécuter toutes les cellules où ont été définies les différentes fonctions (en prénant le soin de ne pas appeler ces fonctions : 
        - *N'exécutez pas les cellules code après les cellules des fonctions*
    - Après vous êtes assurés que toutes les fonctions aient été chargées, exécutez la cellule ci-dessous  *APPLICATIONS : APPLYING FUNCTIONS FOR COLLECT* :
        - *Prenez le soins de bien renseigner les chemins du driver et du répertoire de stockage de vos données.
    - Exécuter la cellule
        
        
- 2. Action N°2 : 
    
    - Ouvrir un nouveau notebook
    - Copier toutes les fonctions définies
    - Puis les mettre dans une seule cellule
        - Ne pas oublier d'y mettre aussi tous les packages
    - Insérer maintenant le code de la cellule ci-dessous dans le nouveau notebook à la suite des fonctions (dans la même cellule) :  
        - Prenez le soins de bien rensiegner les chemins du driver et du répertoire de stockage de vos données.
    - Exécuter la cellule
        

- 3. Actions N°3

    - Créer un nouveau fichier ``.py`` 
    - Copier le contenu de la cellule du nouveau notebook
    - Insérer le contenu dans le fichier ``.py`` créé
    - Exécuter le fichier en invite de commande
    

In [ ]:
def login_to_group(login, password) :
    
    # from acces import email, password
    driver.find_element(By.XPATH, '//*[@id="email"]').send_keys(login)
    driver.find_element(By.XPATH, '//*[@id="pass"]').send_keys(password)
    driver.find_element(By.XPATH, '//*[@id="u_0_d_/e"]').click()

In [ ]:
# ============= APPLICATIONS : APPLYING FUNCTIONS FOR COLLECT ===================#
print("Heure de DEBUT de collecte \t{}".format(str(datetime.now())[:19]))

# ## Group to collact an its mobile URL 
group = 'ucaevd' 
url = 'https://m.facebook.com/groups/consommateurs.abidjan/'

## Complementary element to get POST URL
permalink = 'permalink/'

## Driver Path
# driver_location = "put_your_driver_path_here"
driver_location = "C:/Users/FJPN3406/Documents/FORMATIONS/WEB_SCRAPING/PRATIQUES_WEB_SCRAPING/scraping_env/geckodriver.exe"

## Storage Space for collected data
#dossier = "put_your_root_storage_directory_here"

os.chdir(os.getcwd())
dossier = os.makedirs("./DATA", exist_ok=True)
comments_dir = "commentaires"
posts_dir = "posts"

## Connect to driver and Facebook
driver = connexion_to_group(url, driver_location)

try :

    # Authentification
    try :
        login = "your_login"
        password = "your_password"
        login_to_group(login, password)
    except :
        pass

    ## Get more page
    afficher_plus_de_pubs(2, 3)

    ## Collect publications
    print('\n *********** COLLECTE DES ELEMENTS WEB DES POSTS **********\n ')
    publications = get_all_publications()

    ## Collect authors posts data
    print('\n COLLECTE DES DONNEES DES ********* AUTEURS ********* DES POSTS \n ')
    author_post_data = data_author_posts(publications)

    ## Collect Contents posts data
    print('\n COLLECTE DES ***********  CONTENUS TEXTUELS ********** DES POSTS \n ')
    content_post_data = data_contents_posts(publications)

    ## Collect contents image posts data
    print('\n COLLECTE DES *********** IMAGES *********** DES POSTS \n ')
    sep = "|"
    images_post_data = images_posts_info(publications, sep)

    ## Collect des reactions data
    print('\n  COLLECTE DES *********** REACTIONS *********** DES POSTS \n ')
    reactions_post_data = data_reactions(publications)
except :
    pass
    
## Get All collected posts data
print('\n *********** REGROUPEMENTS DE TOUTES LES DONNEES DES POSTS **********\n ')
posts_data_reacts = post_data_reacts_content(author_post_data, 
                                             content_post_data,
                                             images_post_data,
                                             reactions_post_data)
posts_data_reacts = posts_data_reacts[posts_data_reacts['text_pub']!='']

# stored_path = dossier + group+"\\"+posts_dir+"\\posts_data_reacts"+file_date + ".csv"
print("\n STOCKAGE DES DONNEES DES POSTS \n")
stored_post_data_to_csv(data = posts_data_reacts,
                            dossier = dossier,
                            group_name = group_name,
                            type_infos = 'posts',
                            file_name = 'posts_data_reacts',
                            sep = "@")

print("\n STOCKAGE DES DONNEES DES SENTIMENTS \n")
stored_post_data_to_parquet(data = posts_data_reacts,
                            dossier = dossier,
                            group_name = group_name,
                            type_infos = 'posts',
                            file_name = 'posts_data_reacts')

driver.quit()
print("\nHeure de FIN de collecte \t{}".format(str(datetime.now())[:19]))

Merci d'avoir accordé un si grand intérêt à ce cours!!

Nous restons ouvert pour d'éventuelles préoccupations.

**ON NE FINIT JAMAIS D'APPRENDRE**


**VOS CRITIQUES & SUGGESTIONS SONT LES BIENVENUES POUR AIDER A AMELIORER LE COURS**



  ![alt](images/merci-bonhomme.gif) 